## TSNE Analysis of VirProBERT Few Shot Classifier
### Dataset: EMBL mapping, Vertebrates, IDV

**Models**: VirProBERT, Few-Shot Classifier

**Maximum Sequence Length**: segment len 256, stride 64

In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), "..", "..", "..", "..", ".."))
sys.path.append(os.path.join(os.getcwd(), "..", "..", "..", ".."))
sys.path.append(os.path.join(os.getcwd(), "..", "..", ".."))
sys.path.append(os.path.join(os.getcwd(), "..", ".."))
sys.path

['/home/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/few_shot_learning',
 '/opt/conda/lib/python38.zip',
 '/opt/conda/lib/python3.8',
 '/opt/conda/lib/python3.8/lib-dynload',
 '',
 '/home/blessyantony/.local/lib/python3.8/site-packages',
 '/opt/conda/lib/python3.8/site-packages',
 '/opt/conda/lib/python3.8/site-packages/IPython/extensions',
 '/home/blessyantony/.ipython',
 '/home/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/few_shot_learning/../../../../..',
 '/home/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/few_shot_learning/../../../..',
 '/home/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/few_shot_learning/../../..',
 '/home/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/few_shot_learning/../..']

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

In [3]:
virprobert_embedding_file_path = os.path.join(os.getcwd(), "..", "..", "..", "output/raw/uniref90_embl_vertebrates_idv_all_hosts/20241125/host_multi/embedding/mlm_tfenc_l6_h8_lr1e-4_uniref90viridae_vs30_hybrid_attention_msl256s64ae_bn_cls_fnn_2l_d1024_lr3e-4_output.csv")

In [4]:
virprobert_embedding_df = pd.read_csv(virprobert_embedding_file_path, index_col=0)
virprobert_embedding_df

,0,1,2,3,4,5,6,7,8,9,...,504,505,506,507,508,509,510,511,uniref90_id,y_true
0,2.279394,1.620726,2.579670,-0.963883,-1.920458,-5.949651,-2.563740,-0.026019,0.406225,-1.701671,...,2.919592,-0.253319,-4.889799,2.364681,-3.291519,0.942372,-1.680881,-0.885012,['UniRef90_A0A6B9N6T1'],Homo sapiens
1,2.387317,1.551755,2.425246,-1.072514,-1.832234,-6.161521,-2.461049,-0.039671,0.429921,-1.664472,...,2.922262,-0.285740,-4.780171,2.387689,-3.128775,1.044526,-1.755657,-0.791419,['UniRef90_E5LBB4'],Homo sapiens
2,2.245624,1.562585,2.445451,-1.066500,-1.788550,-5.797822,-2.373322,-0.185615,0.532699,-1.541822,...,2.820489,-0.194701,-4.621329,2.319048,-3.280520,0.885487,-1.659757,-0.850658,['UniRef90_A0A2U8JI99'],Homo sapiens
3,2.428730,1.816946,2.602903,-1.440594,-1.497192,-6.213723,-2.794992,-0.073508,0.924146,-1.397572,...,2.737576,-0.441219,-4.508254,2.753175,-3.573537,0.682276,-1.232581,-0.813810,['UniRef90_I2E6B0'],Homo sapiens
4,2.174568,1.445204,2.535828,-1.076643,-1.795169,-6.003596,-2.423218,-0.196586,0.427582,-1.623530,...,3.043846,-0.210833,-4.797626,2.443953,-3.533220,0.857017,-1.743273,-0.964198,['UniRef90_A0A0B4L2D6'],Homo sapiens
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220063,2.204849,1.462360,2.506810,-1.004451,-1.869652,-5.883219,-2.481225,-0.092410,0.514070,-1.614301,...,2.888274,-0.185870,-4.564615,2.360741,-3.134453,0.923565,-1.574848,-0.809196,['UniRef90_L0BZF6'],Homo sapiens
220064,2.104528,1.260965,2.445144,-0.837480,-1.956248,-5.782840,-2.410522,-0.301919,0.357308,-1.523820,...,2.911995,-0.237620,-4.681486,2.521713,-3.290728,1.033446,-1.751767,-0.908836,['UniRef90_M9Q201'],Homo sapiens
220065,2.517005,1.829799,2.454566,-1.195144,-2.278381,-6.046958,-2.874412,-0.284011,0.838553,-1.448158,...,2.718246,-0.298665,-4.887084,2.896215,-3.320712,1.063822,-1.473011,-0.896093,['UniRef90_L7PEI8'],Homo sapiens
220066,2.094569,1.268337,2.136519,-0.924955,-1.926454,-5.656223,-2.328016,-0.361333,0.290083,-1.461919,...,2.815954,-0.272736,-4.558395,2.479731,-3.095020,1.046725,-1.720842,-0.861289,['UniRef90_D3TXN6'],Homo sapiens


In [5]:
label_col = "y_true"
virprobert_embedding_df[label_col].unique()

array(['Homo sapiens', 'Macaca mulatta', 'Chlorocebus pygerythrus',
       'Cercocebus atys', 'Felis catus', 'Macaca nemestrina',
       'Colobus guereza', 'Mandrillus sphinx', 'Lynx rufus',
       'Chlorocebus sabaeus', 'Cercopithecus solatus',
       'Cercocebus torquatus', 'Panthera leo', 'Felis silvestris catus',
       'Cercopithecus neglectus', 'Macaca arctoides',
       'Cercopithecus cephus', 'Chlorocebus tantalus', 'Felis domesticus',
       'Mandrillus leucophaeus', 'Puma concolor',
       'Cercopithecus nictitans', 'Cercopithecus ascanius', 'Bos taurus',
       'Cercopithecus lhoesti', 'Cercopithecus aethiops',
       'Cercopithecus mona', 'Semnopithecus entellus', 'Panthera pardus',
       'Papio ursinus', 'Cercopithecus erythrotis', 'Macaca fascicularis',
       'Cercopithecus mitis', 'Procolobus verus',
       'Cercocebus torquatus atys', 'Piliocolobus badius',
       'Piliocolobus tephrosceles', 'Cercopithecus aethiops pygerythrus',
       'Miopithecus ogouensis', 'Otoco

In [6]:
def compute_class_distribution(df, label_col, n):
    labels_counts = df[label_col].value_counts()
    print(labels_counts)
    labels_counts = labels_counts.to_dict()
    return labels_counts

In [7]:
metadata = compute_class_distribution(virprobert_embedding_df, label_col, n=virprobert_embedding_df.shape[0])

Homo sapiens                          218631
Macaca mulatta                           849
Felis catus                              143
Cercocebus atys                          122
Puma concolor                             24
Macaca nemestrina                         23
Mandrillus sphinx                         23
Mandrillus leucophaeus                    20
Cercopithecus nictitans                   19
Cercopithecus solatus                     17
Cercocebus torquatus                      16
Cercopithecus aethiops                    14
Chlorocebus sabaeus                       13
Colobus guereza                           13
Cercopithecus cephus                      13
Panthera leo                              12
Cercopithecus neglectus                   12
Cercopithecus mona                        10
Chlorocebus tantalus                      10
Felis silvestris catus                     9
Cercopithecus lhoesti                      8
Chlorocebus pygerythrus                    7
Cercopithe

In [8]:
selected_hosts = [k for k, v in metadata.items() if v >= 6]

In [9]:
selected_hosts

['Homo sapiens',
 'Macaca mulatta',
 'Felis catus',
 'Cercocebus atys',
 'Puma concolor',
 'Macaca nemestrina',
 'Mandrillus sphinx',
 'Mandrillus leucophaeus',
 'Cercopithecus nictitans',
 'Cercopithecus solatus',
 'Cercocebus torquatus',
 'Cercopithecus aethiops',
 'Chlorocebus sabaeus',
 'Colobus guereza',
 'Cercopithecus cephus',
 'Panthera leo',
 'Cercopithecus neglectus',
 'Cercopithecus mona',
 'Chlorocebus tantalus',
 'Felis silvestris catus',
 'Cercopithecus lhoesti',
 'Chlorocebus pygerythrus',
 'Cercopithecus mitis']

In [10]:
len(selected_hosts)

23

In [11]:
import umap

2024-11-26 02:14:59.042101: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
mapper = umap.UMAP().fit(fmnist.data)